[LCS Returns](https://www.hackerrank.com/contests/hourrank-11/challenges/tutzki-and-lcs)
===========================================================================================

The input is a pair of strings, `a` and `b`. The task is to find in how many ways can we add a letter to `a` so that ``LCS(a,b)`` is increased by 1, where ``LCS`` stands for [Longest Common Sequence](https://en.wikipedia.org/wiki/Longest_common_subsequence_problem?oldformat=true).
The basic idea is to count for how many "horisontal" lines are there in the (|a|+1)*(|b|+1) dynamic-programming table of LCS algorithm (just thing about it...). Then we should only count "horizontal" lines that belong to the optimal pathes. Finally, for each ``i = 0,1 ..., |a|`` we should count "horizontal" lines that correspond to distinct letters in ``b`` (this is accounted by ``std::set<std::pair<char, int>>``).
```    
#include <cmath>
#include <cstdio>
#include <vector>
#include <iostream>
#include <algorithm>
#include <map>
#include <sstream>
#include <queue>
using namespace std;

int lcs(const std::string& a, const std::string& b) {
    int la = a.size();
    int lb = b.size();
    
    // Forward path on dynamic programming table
    int* lcs[a.size()+1];
    for (int i = 0; i <= la; i++) {
        lcs[i] = new int[b.size()+1];
        for (int j =0 ; j <= lb;j++)
            lcs[i][j]=0;
    }
    
    for (int i = 0; i < la; i++) {
        for (int j = 0; j < lb; j++) {
            if (a[i] == b[j])
                lcs[i+1][j+1] = lcs[i][j] + 1;
            else
                lcs[i+1][j+1] = std::max(lcs[i][j+1], lcs[i+1][j]);
        }
    }
    
    
    // Backward path on dynamic programming table
    int MAX = 10000;    // used to track that we've already been to the cell before

    std::queue<std::pair<int, int>> tasks;    // cells that need to be processed
    tasks.push(std::make_pair(la, lb));
    
    std::map<std::pair<char, int>, bool> retval;
    
    while (tasks.size() > 0) {
        auto t = tasks.front();
        tasks.pop();
        
        int i = t.first;
        int j = t.second;
        if (lcs[i][j] >= MAX) continue;  // cell has already been processed
        lcs[i][j] += MAX;
        if (i > 0 && j > 0 && a[i-1] == b[j-1])
            tasks.push(std::make_pair(i-1, j-1));
        if (i > 0 && (lcs[i][j]%MAX) == (lcs[i-1][j]%MAX)) {
            tasks.push(std::make_pair(i-1, j));
        }
        if (j > 0 && (lcs[i][j]%MAX) == (lcs[i][j-1]%MAX)) {
            tasks.push(std::make_pair(i, j-1));
            
            // yep, now we found a valid opportunity to increas LCS
            // by inserting a letter b[j-1] between a[i-1] and a[i]; (or in front of a, when i==0)
            retval[std::make_pair(b[j-1], i)] = true;     
        } 
    }

    for (int i = 0; i <= la; i++) 
        delete [] lcs[i];

    return retval.size();
}

int main() {
    string a, b;
    cin >> a;
    cin >> b;
    
    cout << lcs(a, b) << endl;
    
    return 0;
}
```